# Fazi logika

Pogledati [materijale](../../2020_2021/materials/02_fuzzy_logic.html)

In [23]:
from typing import List

In [24]:
class FuzzyInput:
    """Represents one fuzzy input set"""

    def __init__(self, name: str, xs: List[float], ys: List[float], x0: float):
        self.name = name
        self.xs = xs
        self.ys = ys

        self.mu = self.calc_mu(x0)

    def calc_mu(self, x0: float):
        """Calculates the degree of membership to the fuzzy set"""
        if x0 <= self.xs[0]:
            return self.ys[0]
        elif x0 >= self.xs[-1]:
            return self.ys[-1]
        
        for x1, x2, y1, y2 in zip(self.xs[:-1], self.xs[1:], self.ys[:-1], self.ys[1:]):
            if x0 >= x1 and x0 <= x2:
                return y1 + (y2 - y1) / (x2 - x1) * (x0 - x1)


In [25]:
class FuzzyOutput:
    """Represents one fuzzy output set"""
    
    def __init__(self, name: str, xs: List[float], ys: List[float]):
        self.name = name
        self.xs = xs
        self.ys = ys

        # ovo treba da popune FuzzyRule-ovi
        self.mu = 0
        
        # c je reprezentativni predstavnik fazi skupa
        xs_where_y_1 = [x for x, y in zip(xs, ys) if y == 1]
        self.c = sum(xs_where_y_1) / len(xs_where_y_1)

In [26]:
import enum

class LogicOperator(enum.Enum):
    AND = enum.auto()
    OR = enum.auto()
    # TODO za domaci dodati i negaciju

In [27]:
LogicOperator.AND == LogicOperator.OR

False

In [28]:
class FuzzyRule:
    """Fuzzy rule that given two FuzzyInputs and a logic operator calculates degree of membership in FuzzyOutput"""
    def __init__(self, input1: FuzzyInput, input2: FuzzyInput, operator: LogicOperator, output: FuzzyOutput):
        if operator == LogicOperator.AND:
            new_value = min(input1.mu, input2.mu)
        elif operator == LogicOperator.OR:
            new_value = max(input1.mu, input2.mu)
        else:
            raise ValueError(f"Invalid operator: {operator}")
        
        output.mu = max(output.mu, new_value)

In [29]:
list(zip([1,2,3], [4,5,6]))

[(1, 4), (2, 5), (3, 6)]

In [31]:
consumption = []
consumption.append(FuzzyInput('mala', [3,10],[1,0],9))
consumption.append(FuzzyInput('srednja', [7,10,12,15],[0,1,1,0],9))
consumption.append(FuzzyInput('velika', [12,15],[0,1],9))

reliability = []
reliability.append(FuzzyInput('visoka', [5,10],[1,0], 8))
reliability.append(FuzzyInput('niska', [8,15],[0,1], 8))

value = []
value.append(FuzzyOutput('mala', [7,15],[1,0]))
value.append(FuzzyOutput('srednja', [7,15,25,40],[0,1,1,0]))
value.append(FuzzyOutput('velika', [25,40],[0,1]))

rules = []
rules.append(FuzzyRule(consumption[0], reliability[0], LogicOperator.AND, value[2]))
rules.append(FuzzyRule(consumption[0], reliability[1], LogicOperator.AND, value[1]))
rules.append(FuzzyRule(consumption[1], reliability[0], LogicOperator.AND, value[1]))
rules.append(FuzzyRule(consumption[1], reliability[1], LogicOperator.AND, value[1]))
rules.append(FuzzyRule(consumption[2], reliability[0], LogicOperator.AND, value[1]))
rules.append(FuzzyRule(consumption[2], reliability[1], LogicOperator.AND, value[0]))

In [33]:
# provera da li FuzzyRule radi ono sto treba
for v in value:
    print(f"mu({v.name}) = {v.mu}")

mu(mala) = 0
mu(srednja) = 0.3999999999999999
mu(velika) = 0.1428571428571429


In [35]:
def defuzzify(outputs: List[FuzzyOutput]):
    """Defuzzification - using weighted average"""
    numerator = sum(o.mu * o.c for o in outputs)
    denominator = sum(o.mu for o in outputs)
    return numerator / denominator

In [36]:
defuzzify(value)

25.263157894736846